In [ ]:
#Automated Readability Index (ARI)
#Calculation:
# 4.71 (Character / Word) + 0.5(Word / Sentence) - 21.43
from nltk.tokenize import sent_tokenize
#number_of_sentences = sent_tokenize(sentences)
#countOfWords = len("Geeksforgeeks is best Computer Science Portal".split())
#print("Count of Words in the given Sentence:", countOfWords)
from textblob import TextBlob




In [ ]:
from nltk.tokenize import sent_tokenize
import datetime
def ARI(paragraph):
    countofwords = len(paragraph.split())
    numofsentences = len(sent_tokenize(paragraph))
    if countofwords == 0:
        countofwords = 1
    if numofsentences == 0:
        numofsentences = 1
    ari = 4.71 * len(paragraph) / countofwords + 0.5 * countofwords / numofsentences -21.43
    return ari
def converttimestamp(timestamp_string):
    obj = datetime.datetime.strptime(timestamp_string, '%a %b %d %H:%M:%S %z %Y')
    return obj

def calc_timedelta(ts1, ts2):
    ts1_obj = converttimestamp(ts1)
    ts2_obj = converttimestamp(ts2)
    return (ts1_obj - ts2_obj).days

In [ ]:
# High level feature extractor. 
# We originally split these features into two types, ARI (text features) and UGP (user profile features)
import os
import json
import signal
import math
import preprocessor as p
from nltk.sentiment import SentimentIntensityAnalyzer
import numpy as np
p.set_options(p.OPT.URL, p.OPT.EMOJI)
sia = SentimentIntensityAnalyzer()
import fasttext
import pickle
import numpy as np
model = fasttext.load_model()
file = open('', 'rb')
svm = pickle.load(file)
def get_doc_vector_fasttext(document, model):
    vector_document = [model[i] for i in document.split() if i in model.words]
    document_to_vector = np.sum(vector_document, axis=0)
    return document_to_vector
#import preprocessing as p
def get_prediction_misinfo(document, model_fasttext, model_svm):
    vector_document = [model_fasttext[i] for i in document.split() if i in model_fasttext.words]
    try:
        document_to_vector = np.sum(vector_document, axis=0)  
        return svm.predict([document_to_vector])[0]
    except Exception as e:
        return 0
def cal_avg_ARI_all_split(path, keyword):
    final_result = {}
    in_path = os.path.join(path,keyword)
    for each in os.listdir(in_path):
        result = cal_avg_ARI_day_split(os.path.join(in_path, each), 4)
        for each_result in result:
            final_string = each + "_" + each_result
            re = [result[each_result]["total_ARI"], result[each_result]["total_pos"], result[each_result]["total_neg"], result[each_result]["total_neu"], 
                  result[each_result]["total_comp"],result[each_result]["total_subj"], result[each_result]["total_pol"], result[each_result]["ARI_sd"],
                  result[each_result]["pos_sd"], result[each_result]["neg_sd"], result[each_result]["neu_sd"], result[each_result]["comp_sd"], 
                  result[each_result]["subj_sd"], result[each_result]["pol_sd"], result[each_result]["avg_part"], result[each_result]["part_sd"],
                  result[each_result]["avg_misinfo"], result[each_result]["total_para"]]
            final_result[final_string] = re
    #Write to file
    __PATH__ = ""
    
    json_object = json.dumps(final_result, indent = 4) 
    with open(os.path.join(__PATH__, keyword) + ".json", "w") as outfile:
        outfile.write(json_object)
    outfile.close()    
def cal_avg_UGP_all_split(path, keyword):
    final_result = {}
    in_path = os.path.join(path,keyword)
    for each in os.listdir(in_path):
        result = cal_avg_UGP_day_split(os.path.join(in_path, each), 4)
        for each_result in result:
            final_string = each + "_" + each_result
            re = [result[each_result]["total_verified"], result[each_result]["total_geo_enabled"], result[each_result]["total_fol_count"], 
                  result[each_result]["total_fri_count"], result[each_result]["total_stat_count"], result[each_result]["total_fav_count"],
                  result[each_result]["total_list_created"],result[each_result]["total_account_age"], result[each_result]["total_UGP_word_count"], 
                  result[each_result]["total_screen_name_length"],result[each_result]["total_UGP_ARI"], result[each_result]["fol_count_sd"],
                  result[each_result]["fri_count_sd"], result[each_result]["stat_count_sd"],result[each_result]["fav_count_sd"], 
                  result[each_result]["list_created_sd"], result[each_result]["account_age_sd"], result[each_result]["word_count_sd"],
                  result[each_result]["name_length_sd"], result[each_result]["UGP_ARI_sd"], result[each_result]["total_para"]]
            final_result[final_string] = re
    #Write to file
    __PATH__ = "/home/ps/luyue/UGP_RESULT"
    
    json_object = json.dumps(final_result, indent = 4) 
    with open(os.path.join(__PATH__, keyword) + ".json", "w") as outfile:
        outfile.write(json_object)
    outfile.close()
def cal_avg_ARI_day_split(day_path, split_num = 4):
    #Split num must be dividable by 24.
    extract_data = {}
    for i in range(0, split_num):
        string = "slot_" + str(i)
        extract_data[string] = {"total_ARI": 0,"total_pos": 0, "total_neg": 0, "total_neu" : 0, "total_comp" : 0, 
            "total_subj": 0, "total_pol": 0, "ARI_sd" : 0, "pos_sd" : 0, "neg_sd" : 0, "total_misinfo": 0,
            "neu_sd": 0, "comp_sd": 0, "subj_sd": 0, "pol_sd": 0, "avg_part": 0, 
            "part_sd": 0, "avg_misinfo": 0, "total_para": 0, "ARI_list" : [], "pos_list": [], "neg_list": [], "neu_list": [], "comp_list": [],
            "subj_list": [], "pol_list": [], "message_list": {}, "participation_count_list": []}
    for each in os.listdir(day_path):
        with open(os.path.join(day_path, each), 'r', encoding = 'utf-8') as f:
            try:
                listdict = splitdict(f.read())
            except Exception as e:
                listdict = []
            signal.alarm(180)
            try:
                for each in listdict:
                    timestamp = converttimestamp(each['created_at'])
                    slot = math.floor(timestamp.hour / 6)
                    slot_string = "slot_" + str(slot)
                    if 'retweeted_status' in each.keys():
                        st = p.clean(each['retweeted_status']["full_text"].lower())
                    else:
                        st = p.clean(each["full_text"].lower())
                    st2 = p.clean(each['user']["description"].lower())    
                    blob = TextBlob(st)
                    x = sia.polarity_scores(st)
                    extract_data[slot_string]["total_pos"] += x["pos"]
                    extract_data[slot_string]["total_neg"] += x["neg"]
                    extract_data[slot_string]["total_neu"] += x["neu"]
                    extract_data[slot_string]["total_comp"] += x["compound"]
                    extract_data[slot_string]["total_subj"] += blob.sentiment.subjectivity
                    extract_data[slot_string]["total_pol"] += blob.sentiment.polarity
                    extract_data[slot_string]["total_misinfo"] += get_prediction_misinfo(st, model, svm)
                    x2 = ARI(st)
                    extract_data[slot_string]["total_ARI"] += x2
                    extract_data[slot_string]["total_para"] += 1
                    extract_data[slot_string]["ARI_list"].append(x2)
                    extract_data[slot_string]["pos_list"].append(x["pos"])
                    extract_data[slot_string]["neg_list"].append(x["neg"])
                    extract_data[slot_string]["neu_list"].append(x["neu"])
                    extract_data[slot_string]["comp_list"].append(x["compound"])
                    extract_data[slot_string]["subj_list"].append(blob.sentiment.subjectivity)
                    extract_data[slot_string]["pol_list"].append(blob.sentiment.polarity)
                    if st in extract_data[slot_string]["message_list"]:
                        extract_data[slot_string]["message_list"][st] += 1
                    else:
                        extract_data[slot_string]["message_list"][st] = 1         
            except Exception as e:
                print(e)
            signal.alarm(0)
            f.close()
    for slot_string in extract_data:    
        for x in extract_data[slot_string]["message_list"]:
            extract_data[slot_string]["participation_count_list"].append(extract_data[slot_string]["message_list"][x]) 
        extract_data[slot_string]["avg_part"] = extract_data[slot_string]["total_para"] / (len(extract_data[slot_string]["participation_count_list"]) + 1)
        extract_data[slot_string]["avg_misinfo"] = extract_data[slot_string]["total_misinfo"] / (extract_data[slot_string]["total_para"] + 1)
        extract_data[slot_string]["part_sd"] = np.std(extract_data[slot_string]["participation_count_list"])
        extract_data[slot_string]["ARI_sd"] = np.std(extract_data[slot_string]["ARI_list"])
        extract_data[slot_string]["pos_sd"] = np.std(extract_data[slot_string]["pos_list"])
        extract_data[slot_string]["neg_sd"] = np.std(extract_data[slot_string]["neg_list"])
        extract_data[slot_string]["neu_sd"] = np.std(extract_data[slot_string]["neu_list"])
        extract_data[slot_string]["comp_sd"] = np.std(extract_data[slot_string]["comp_list"])
        extract_data[slot_string]["subj_sd"] = np.std(extract_data[slot_string]["subj_list"])
        extract_data[slot_string]["pol_sd"] = np.std(extract_data[slot_string]["pol_list"])
    return extract_data
def cal_avg_UGP_day_split(day_path, split_num = 4):
    extract_data = {}
    for i in range(0, split_num):
        string = "slot_" + str(i)
        extract_data[string] = {"total_verified": 0, "total_geo_enabled": 0, "total_fol_count": 0, "total_fri_count": 0, "total_stat_count": 0, 
           "total_fav_count": 0, "total_list_created": 0,"total_account_age": 0, "total_UGP_word_count": 0, "total_screen_name_length": 0,
           "total_UGP_ARI": 0, "fol_count_sd": 0, "fri_count_sd": 0, "stat_count_sd": 0,"fav_count_sd": 0, 
           "list_created_sd": 0, "account_age_sd": 0, "word_count_sd": 0, "name_length_sd": 0, "UGP_ARI_sd": 0, 
           "total_para": 0, "UGP_ARI_list": [], "name_length_list": [], "word_count_list": [], "account_age_list": [], "list_created_list": [],
           "fav_count_list": [], "stat_count_list": [], "fri_count_list": [], "fol_count_list": []}
    for each in os.listdir(day_path):
        with open(os.path.join(day_path, each), 'r', encoding = 'utf-8') as f:
            try:
                listdict = splitdict(f.read())
            except Exception as e:
                listdict = []
            signal.alarm(180)
            try:
                for each in listdict:
                    timestamp = converttimestamp(each['created_at'])
                    slot = math.floor(timestamp.hour / 6)
                    slot_string = "slot_" + str(slot)                    
                    st = p.clean(each['user']["description"].lower())
                    UGP_ARI = ARI(st)
                    #blob = TextBlob(st)
                    screen_name_length = len(each['user']['screen_name'])
                    UGP_word_count = len(each['user']['description'].split())
                    account_age = calc_timedelta(each['created_at'], each['user']['created_at'])
                    list_created = each['user']['listed_count']
                    fav_count = each['user']['favourites_count']
                    stat_count = each['user']['statuses_count']
                    fri_count = each['user']['friends_count']
                    fol_count = each['user']['followers_count']
                    if each['user']['geo_enabled']:
                        geo_enable = 1
                    else:
                        geo_enable = 0
                    if each['user']['verified']:
                        verified = 1
                    else:
                        verified = 0
                    extract_data[slot_string]["total_UGP_ARI"] += UGP_ARI
                    extract_data[slot_string]["total_verified"] += verified
                    extract_data[slot_string]["total_geo_enabled"] += geo_enable
                    extract_data[slot_string]["total_fol_count"] += fol_count
                    extract_data[slot_string]["total_fri_count"] += fri_count
                    extract_data[slot_string]["total_stat_count"] += stat_count
                    extract_data[slot_string]["total_fav_count"] += fav_count
                    extract_data[slot_string]["total_list_created"] += list_created
                    extract_data[slot_string]["total_account_age"] += account_age
                    extract_data[slot_string]["total_UGP_word_count"] += UGP_word_count
                    extract_data[slot_string]["total_screen_name_length"] += screen_name_length
                    extract_data[slot_string]["total_para"] += 1
                    extract_data[slot_string]["UGP_ARI_list"].append(UGP_ARI)
                    extract_data[slot_string]["name_length_list"].append(screen_name_length)
                    extract_data[slot_string]["word_count_list"].append(UGP_word_count)
                    extract_data[slot_string]["account_age_list"].append(account_age)
                    extract_data[slot_string]["list_created_list"].append(list_created)
                    extract_data[slot_string]["fav_count_list"].append(fav_count)
                    extract_data[slot_string]["stat_count_list"].append(stat_count)
                    extract_data[slot_string]["fri_count_list"].append(fri_count)
                    extract_data[slot_string]["fol_count_list"].append(fol_count)
            except Exception as e:
                print(e)
            signal.alarm(0)
            f.close()
        extract_data[slot_string]["UGP_ARI_sd"] = np.std(extract_data[slot_string]["UGP_ARI_list"])
        extract_data[slot_string]["name_length_sd"] = np.std(extract_data[slot_string]["name_length_list"])
        extract_data[slot_string]["word_count_sd"] = np.std(extract_data[slot_string]["word_count_list"])
        extract_data[slot_string]["account_age_sd"] = np.std(extract_data[slot_string]["account_age_list"])
        extract_data[slot_string]["list_created_sd"] = np.std(extract_data[slot_string]["list_created_list"])
        extract_data[slot_string]["fav_count_sd"] = np.std(extract_data[slot_string]["fav_count_list"])
        extract_data[slot_string]["stat_count_sd"] = np.std(extract_data[slot_string]["stat_count_list"])
        extract_data[slot_string]["fri_count_sd"] = np.std(extract_data[slot_string]["fri_count_list"])
        extract_data[slot_string]["fol_count_sd"] = np.std(extract_data[slot_string]["fol_count_list"])
    #print("Finished:" + day_path)
    return extract_data
def cal_avg_ARI_day(day_path):
    total_ARI = 0
    total_para = 0
    total_pos = 0
    total_neg = 0
    total_neu = 0
    total_comp = 0
    total_subj = 0
    total_pol = 0
    total_misinfo = 0
    ARI_list = []
    pos_list = []
    neg_list = []
    neu_list = []
    comp_list = []
    message_list = {}
    participation_count_list = []
    subj_list = []
    pol_list = []
    for each in os.listdir(day_path):
        with open(os.path.join(day_path, each), 'r', encoding = 'utf-8') as f:
            try:
                listdict = splitdict(f.read())
            except Exception as e:
                listdict = []
            signal.alarm(180)
            try:
                for each in listdict:
                    if 'retweeted_status' in each.keys():
                        st = p.clean(each['retweeted_status']["full_text"].lower())
                    else:
                        st = p.clean(each["full_text"].lower())
                    st2 = p.clean(each['user']["description"].lower())
                    blob = TextBlob(st)
                    x = sia.polarity_scores(st)
                    total_pos += x["pos"]
                    total_neg += x["neg"]
                    total_neu += x["neu"]
                    total_comp += x["compound"]
                    total_subj += blob.sentiment.subjectivity
                    total_pol += blob.sentiment.polarity
                    total_misinfo += get_prediction_misinfo(st, model, svm)
                    x2 = ARI(st)
                    total_ARI += x2
                    total_para += 1
                    ARI_list.append(x2)
                    pos_list.append(x["pos"])
                    neg_list.append(x["neg"])
                    neu_list.append(x["neu"])
                    comp_list.append(x["compound"])
                    subj_list.append(blob.sentiment.subjectivity)
                    pol_list.append(blob.sentiment.polarity)
                    if st in message_list:
                        message_list[st] += 1
                    else:
                        message_list[st] = 1
                    #message_list[st] = 1
            except Exception as e:
                print(e)
            signal.alarm(0)
            f.close()
    for x in message_list:
        participation_count_list.append(message_list[x])
    avg_part = total_para / (len(participation_count_list) + 1)
    avg_misinfo = total_misinfo / (total_para + 1)
    part_sd = np.std(participation_count_list)
    ARI_sd = np.std(ARI_list)
    pos_sd = np.std(pos_list)
    neg_sd = np.std(neg_list)
    neu_sd = np.std(neu_list)
    comp_sd = np.std(comp_list)
    subj_sd = np.std(subj_list)
    pol_sd = np.std(pol_list)
    return [total_ARI,total_pos, total_neg, total_neu, total_comp, 
            total_subj, total_pol, ARI_sd, pos_sd, neg_sd, 
            neu_sd, comp_sd, subj_sd, pol_sd, avg_part, 
            part_sd,avg_misinfo, total_para]
def cal_avg_UGP_day(day_path):
    total_verified = 0
    total_geo_enabled = 0
    total_fol_count = 0
    total_fri_count = 0
    total_stat_count = 0
    total_fav_count = 0
    total_list_created = 0
    total_account_age = 0
    total_UGP_word_count = 0
    total_screen_name_length = 0
    total_UGP_ARI = 0
    total_para = 0
    UGP_ARI_list = []
    name_length_list = []
    word_count_list = []
    account_age_list = []
    list_created_list = []
    fav_count_list = []
    stat_count_list = []
    fri_count_list = []
    fol_count_list = []
    for each in os.listdir(day_path):
        with open(os.path.join(day_path, each), 'r', encoding = 'utf-8') as f:
            try:
                listdict = splitdict(f.read())
            except Exception as e:
                listdict = []
            signal.alarm(180)
            try:
                for each in listdict:
                    st = p.clean(each['user']["description"].lower())
                    UGP_ARI = ARI(st)
                    #blob = TextBlob(st)
                    screen_name_length = len(each['user']['screen_name'])
                    UGP_word_count = len(each['user']['description'].split())
                    account_age = calc_timedelta(each['created_at'], each['user']['created_at'])
                    list_created = each['user']['listed_count']
                    fav_count = each['user']['favourites_count']
                    stat_count = each['user']['statuses_count']
                    fri_count = each['user']['friends_count']
                    fol_count = each['user']['followers_count']
                    if each['user']['geo_enabled']:
                        geo_enable = 1
                    else:
                        geo_enable = 0
                    if each['user']['verified']:
                        verified = 1
                    else:
                        verified = 0
                    total_UGP_ARI += UGP_ARI
                    total_verified += verified
                    total_geo_enabled += geo_enable
                    total_fol_count += fol_count
                    total_fri_count += fri_count
                    total_stat_count += stat_count
                    total_fav_count += fav_count
                    total_list_created += list_created
                    total_account_age += account_age
                    total_UGP_word_count += UGP_word_count
                    total_screen_name_length += screen_name_length
                    total_para += 1
                    UGP_ARI_list.append(UGP_ARI)
                    name_length_list.append(screen_name_length)
                    word_count_list.append(UGP_word_count)
                    account_age_list.append(account_age)
                    list_created_list.append(list_created)
                    fav_count_list.append(fav_count)
                    stat_count_list.append(stat_count)
                    fri_count_list.append(fri_count)
                    fol_count_list.append(fol_count)
            except Exception as e:
                print(e)
            signal.alarm(0)
            f.close()
    UGP_ARI_sd = np.std(UGP_ARI_list)
    name_length_sd = np.std(name_length_list)
    word_count_sd = np.std(word_count_list)
    account_age_sd = np.std(account_age_list)
    list_created_sd = np.std(list_created_list)
    fav_count_sd = np.std(fav_count_list)
    stat_count_sd = np.std(stat_count_list)
    fri_count_sd = np.std(fri_count_list)
    fol_count_sd = np.std(fol_count_list)
    #print("Finished:" + day_path)
    return[total_verified, total_geo_enabled, total_fol_count, total_fri_count, total_stat_count, 
           total_fav_count, total_list_created,total_account_age, total_UGP_word_count, total_screen_name_length,
           total_UGP_ARI, fol_count_sd, fri_count_sd, stat_count_sd,fav_count_sd, 
           list_created_sd, account_age_sd, word_count_sd, name_length_sd, UGP_ARI_sd, 
           total_para]
def cal_avg_ARI_all(path, keyword):
    result = {}
    in_path = os.path.join(path,keyword)
    for each in os.listdir(in_path):
        result[each] = cal_avg_ARI_day(os.path.join(in_path, each))
    #Write to file
    __PATH__ = "/home/ps/luyue/ARI_RESULT_DTM"
    
    json_object = json.dumps(result, indent = 4) 
    with open(os.path.join(__PATH__, keyword) + ".json", "w") as outfile:
        outfile.write(json_object)
    outfile.close()
def cal_avg_ARI_DTM(main_path, keyword):
    __PATH__ = "/home/ps/luyue/ARI_RESULT_DTM"    
    result = {}
    keyword_path = os.path.join(main_path, keyword)
    for each_day in os.listdir(keyword_path):
        day_path = os.path.join(keyword_path, each_day)
        for each_topic in os.listdir(day_path):
            topic_path = os.path.join(day_path, each_topic)
            result_string = keyword + "_" + str(each_day) + "_" + str(each_topic)
            result[result_string] = cal_avg_ARI_day(topic_path)
    json_object = json.dumps(result, indent = 4)
    with open(os.path.join(__PATH__, keyword) + ".json", "w") as outfile:
        outfile.write(json_object)
    outfile.close()
def cal_avg_UGP_DTM(main_path, keyword):
    __PATH__ = "/home/ps/luyue/UGP_RESULT_DTM"    
    result = {}
    keyword_path = os.path.join(main_path, keyword)
    for each_day in os.listdir(keyword_path):
        day_path = os.path.join(keyword_path, each_day)
        for each_topic in os.listdir(day_path):
            topic_path = os.path.join(day_path, each_topic)
            result_string = keyword + "_" + str(each_day) + "_" + str(each_topic)
            result[result_string] = cal_avg_UGP_day(topic_path)
    json_object = json.dumps(result, indent = 4)
    with open(os.path.join(__PATH__, keyword) + ".json", "w") as outfile:
        outfile.write(json_object)
    outfile.close()
def cal_avg_UGP_all(path, keyword):
    result = {}
    in_path = os.path.join(path,keyword)
    for each in os.listdir(in_path):
        result[each] = cal_avg_UGP_day(os.path.join(in_path, each))
    #Write to file
    __PATH__ = "/home/ps/luyue/UGP_RESULT_DTM"
    
    json_object = json.dumps(result, indent = 4) 
    with open(os.path.join(__PATH__, keyword) + ".json", "w") as outfile:
        outfile.write(json_object)
    outfile.close()
def cal_avg_ARI_keyword(path, keyword):
    result = {}
    in_path = os.path.join(path, keyword)
    for each_day in os.listdir(in_path):
        day_path = os.path.join(in_path, day_path)
        result[each_day] = {}
        for each_topic in os.listdir(day_path):
            result[each_day][each_topic] = cal_avg_ARI_day(os.path.join(day_path, each_topic))
def splitdict(string):
    i = 0
    dict_list = []
    for index in range(len(string) - 2):
        if string[index] == "}" and string[index+1] == "{" and string[index+2].isalpha() == False:
            sub_string = string[i:index+1]
            i = index + 1
            try:
                dict1 = json.loads(sub_string)
                dict_list.append(dict1)
            except Exception as e:
                dict1 = []
                #print(sub_string)
                print("ERROR!")
                print(e)
                pass
    sub_string = string[i:]
    dict2 = json.loads(sub_string)
    dict_list.append(dict2)
    return dict_list




In [ ]:
proc_list = []
dir1 = "*" #Directory for your separated topic posts
import multiprocessing
proc_list = []
for each in os.listdir(dir1):
    proc = multiprocessing.Process(target = cal_avg_ARI_DTM, args = (dir1, each, ))
    proc_list.append(proc)
    proc.start()
for process in proc_list:
    process.join()
proc_list = []
for each in os.listdir(dir1):
    proc = multiprocessing.Process(target = cal_avg_UGP_DTM, args = (dir1, each, ))
    proc_list.append(proc)
    proc.start()
for process in proc_list:
    process.join()